In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
xinzone_surface_crack_path = kagglehub.dataset_download('xinzone/surface-crack')

print('Data source import complete.')

# Install SimpleITK
# !pip install SimpleITK

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import SimpleITK as sitk # Now this line should work
import cv2 as cv
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

d:\FinalPBLProj\crackENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data source import complete.


In [ ]:
# Define the training and validation base directories
# data_dir = '/kaggle/working/surface-crack' # Incorrect: Assuming a specific path
data_dir = xinzone_surface_crack_path  # Correct: Use the path returned by kagglehub

# Update train, validation, and test directories to point to the downloaded data
train_dir = os.path.join(data_dir, 'train') # Assuming 'train' folder is inside 'surface-crack'
validation_dir = os.path.join(data_dir, 'valid') # Assuming 'valid' folder is inside 'surface-crack'
test_dir = os.path.join(data_dir, 'test') # Assuming 'test' folder is inside 'surface-crack'

# Directory with training positive pictures
train_positive_dir = os.path.join(train_dir, 'Positive')
# Directory with training negative pictures
train_negative_dir = os.path.join(train_dir, 'Negative')
# Directory with validation positive pictures
validation_positive_dir = os.path.join(validation_dir, 'Positive')
# Directory with validation negative pictures
validation_negative_dir = os.path.join(validation_dir, 'Negative')
# Directory with test positive pictures
test_positive_dir = os.path.join(test_dir, 'Positive')
# Directory with test negative pictures
test_negative_dir = os.path.join(test_dir, 'Negative')
# Check the number of images for each class and set
print(f"There are {len(os.listdir(train_positive_dir))} images of crack for training.\n")
print(f"There are {len(os.listdir(train_negative_dir))} images of no crack for training.\n")
print(f"There are {len(os.listdir(validation_positive_dir))} images of crack for validation.\n")
print(f"There are {len(os.listdir(validation_negative_dir))} images of no crack for validation.\n")
print(f"There are {len(os.listdir(test_positive_dir))} images of crack for test.\n")
print(f"There are {len(os.listdir(test_negative_dir))} images of no crack for test.\n")

## Now take a look at a sample image of each one of the classes:

In [ ]:
print("Sample crack image:")
plt.imshow(load_img(f"{os.path.join(train_positive_dir, os.listdir(train_positive_dir)[0])}"))
plt.show()

print("\nSample no crack image:")
plt.imshow(load_img(f"{os.path.join(train_negative_dir, os.listdir(train_negative_dir)[0])}"))
plt.show()

In [ ]:
# Load the first example of a crack
sample_image  = load_img(f"{os.path.join(train_positive_dir, os.listdir(train_positive_dir)[0])}")

# Convert the image into its numpy array representation
sample_array = img_to_array(sample_image)

print(f"Each image has shape: {sample_array.shape}")

### As expected, the sample image has a resolution of 224x224 and the last dimension is used for each one of the RGB channels to represent color.

# Preprocessing the images

In [ ]:
image_size = 224       #resize all images to 224*224

labels = ['Positive', 'Negative']          #labels from the folders

In [ ]:
def create_training_data(data_dir):              #creating the training data

    images = []

    for label in labels:
        dir = os.path.join(data_dir,label)
        class_num = labels.index(label)

        for image in os.listdir(dir):    #going through all the images in different folders and resizing them

            image_read = cv.imread(os.path.join(dir,image),cv.IMREAD_COLOR)
            image_resized = cv.resize(image_read,(image_size,image_size))
            images.append([image_resized,class_num])

    return np.array(images)

In [ ]:
def create_training_data(data_dir):              #creating the training data

    images = []
    labels_list = []  # Create a separate list for labels

    for label in labels:
        dir = os.path.join(data_dir,label)
        class_num = labels.index(label)

        for image in os.listdir(dir):    #going through all the images in different folders and resizing them

            image_read = cv.imread(os.path.join(dir,image),cv.IMREAD_COLOR)
            image_resized = cv.resize(image_read,(image_size,image_size))
            images.append(image_resized) # Append only image data to images list
            labels_list.append(class_num)  # Append label to labels_list

    return np.array(images), np.array(labels_list) # Return both arrays

X_train, y_train = create_training_data(os.path.join(xinzone_surface_crack_path, 'train'))
X_test, y_test = create_training_data(os.path.join(xinzone_surface_crack_path, 'test'))
X_valid, y_valid = create_training_data(os.path.join(xinzone_surface_crack_path, 'valid'))


### Loading the train and valid Images and Labels together

In [ ]:
X = []
y = []

# Assign the output of create_training_data for the training set to the train variable
# The create_training_data function returns two values: images and labels
X_train, y_train = create_training_data(os.path.join(xinzone_surface_crack_path, 'train'))

# Instead of iterating through 'train', use the returned values directly
X = X_train
y = y_train


X_valid = []
y_valid = []

# Assign the output of create_training_data for the validation set to the valid variable
# The create_training_data function returns two values: images and labels
X_valid, y_valid = create_training_data(os.path.join(xinzone_surface_crack_path, 'valid'))

# Instead of iterating through 'valid', use the returned values directly
#X_valid = X_valid_data
#y_valid = y_valid_data

In [ ]:
X_train = np.array(X).reshape(-1, image_size, image_size, 3)
y_train = np.array(y)
y_train = np.expand_dims(y_train, axis =1)

In [ ]:
X_vald = np.array(X_valid).reshape(-1, image_size, image_size, 3)
y_vald = np.array(y_valid)
y_vald = np.expand_dims(y_vald, axis =1)

### Loading the test Images and Labels together

In [ ]:
X_new = []
y_new = []

# Iterate through X_test and y_test using zip
for feature, label in zip(X_test, y_test):
    X_new.append(feature)          #appending all images
    y_new.append(label)            #appending all labels

In [ ]:
X_test = np.array(X_new).reshape(-1, image_size, image_size, 3)
y_test = np.array(y_new)
y_test = np.expand_dims(y_test, axis =1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train = X_train / 255            # normalizing
X_test = X_test / 255
X_vald = X_vald/ 255

# Transfer learning - Create the pre-trained model

### Download the inception V3 weights into the /tmp/ directory:

In [ ]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

### Now load the InceptionV3 model and save the path to the weights we just downloaded:

In [ ]:
# Import the inception model
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = 'C:\\Users\\Samarth\\Downloads\\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
# create_pre_trained_model
def create_pre_trained_model(local_weights_file):
    pre_trained_model = InceptionV3(input_shape = (224, 224, 3),
                                  include_top = False,
                                  weights = None)

    pre_trained_model.load_weights(local_weights_file)

  # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

  ### END CODE HERE

    return pre_trained_model

# Compiling and viewing the model summary

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)

# Print the model summary
pre_trained_model.summary()

### Pipelining the pre-trained model with your own

In [ ]:
def output_of_last_layer(pre_trained_model):
    last_desired_layer = pre_trained_model.get_layer('mixed7')
    # Access output shape after building the model
    # Build the model with dummy input to resolve the shape
    pre_trained_model.build(input_shape=(None, 224, 224, 3))
    # Get output shape from the output tensor
    print('last layer output shape: ', last_desired_layer.output.shape)
    last_output = last_desired_layer.output
    print('last layer output: ', last_output)

    return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

In [ ]:
# Print the type of the pre-trained model
print(f"The pretrained model has type: {type(pre_trained_model)}")

In [ ]:

def create_final_model(pre_trained_model, last_output):
    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)



    # Add a fully connected layer with 512 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
#     x = layers.Dense(512, activation='relu')(x)
#     # Add a dropout rate of 0.3
#     x = layers.Dropout(0.3)(x)
#     x = layers.Dense(512, activation='relu')(x)
#     # Add a dropout rate of 0.1
#     x = layers.Dropout(0.1)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(1, activation='sigmoid')(x)

    # Create the complete model by using the Model class
    model = Model(pre_trained_model.input, x)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate = 0.0001),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

    ### END CODE HERE

    return model

In [ ]:
# Save your model in a variable
model = create_final_model(pre_trained_model, last_output)

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

# Data Augmentation

In [ ]:
batch_size = 4

train_gen = ImageDataGenerator(rotation_range=10,
                                   horizontal_flip = True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rescale=1.,
                                   zoom_range=0.2,
                                   fill_mode='nearest',
                                   cval=0)

train_generator = train_gen.flow(X_train,y_train,batch_size)
steps_per_epoch = X_train.shape[0]//batch_size

In [ ]:
checkpoint = ModelCheckpoint('surface-crack.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
r = model.fit(train_generator, validation_data=(X_vald, y_vald), steps_per_epoch = steps_per_epoch, epochs= 14,
                       callbacks = [checkpoint])

In [ ]:
# model.save('my_model.h5');

# Plots

In [ ]:
import matplotlib.pyplot as plt
acc = r.history['accuracy']
val_acc = r.history['val_accuracy']
loss = r.history['loss']
val_loss = r.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()

In [ ]:
# new_model = tf.keras.models.load_model('surface-crack')   #loading model to train further

In [ ]:
# new_model.compile(optimizer = Adam(learning_rate = 0.00001), loss = 'binary_crossentropy', metrics = ['accuracy'])

# checkpoint1 = ModelCheckpoint('surface-crack', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

# batch_size = 4

# r1 = new_model.fit(train_generator, validation_data=(X_vald, y_vald), steps_per_epoch = steps_per_epoch, epochs= 10,
#                        callbacks = [checkpoint1])

In [3]:
final_model = tf.keras.models.load_model('surface-crack\content\surface-crack.keras')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\comp\AppData\Local\Temp\ipykernel_8496\3818127217.py:1: SyntaxWarning: invalid escape sequence '\c'
  final_model = tf.keras.models.load_model('surface-crack\content\surface-crack.keras')


In [5]:
pred = final_model.predict(X_test)
pred


NameError: name 'X_test' is not defined

In [ ]:
pred_final = np.where(pred>0.5,1,0)
pred_final

# Evaluating Metrics

In [ ]:
# Get the confusion matrix
CM = confusion_matrix(y_test, pred_final)

fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(8,8))
plt.title('Confusion matrix')
plt.xticks(range(2), ['Positive','Negative'], fontsize=10)
plt.yticks(range(2), ['Positive','Negative'], fontsize=10)
plt.show()

In [ ]:
def perf_measure(y_test, pred_final):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(pred_final)):
        if y_test[i]==pred_final[i]==1:
           TP += 1
        if y_test[i]==1 and y_test[i]!=pred_final[i]:
           FP += 1
        if y_test[i]==pred_final[i]==0:
           TN += 1
        if y_test[i]==0 and y_test[i]!=pred_final[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
tp, fp, tn ,fn = perf_measure(y_test,pred_final)

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f_score = (2*precision*recall)/(precision+recall)

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F-Score is {:.2f}".format(f_score))

In [ ]:
def predict_input_image(img):
    img_4d=img.reshape(-1,224,224,3)
    img_4d=img_4d/255
    prediction = np.zeros((1,2))
    prediction[0,0] = (1-final_model.predict(img_4d)[0,0])
    prediction[0,1] = (final_model.predict(img_4d)[0,0])
    labels = ['crack detected', 'crack is not present']
    return {labels[i]: float(prediction[0,i]) for i in range(2)}

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import img_to_array

# Load trained model
final_model = tf.keras.models.load_model("surface-crack.keras")

# Get the last convolutional layer
last_conv_layer_name = "conv2d_69"  # Change this if needed


# Function to compute Grad-CAM heatmap
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap, predicted_class.numpy()

# Function to apply segmentation on heatmap
def segment_crack(img, heatmap):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)  # Scale heatmap to 0-255

    # No need for cvtColor, heatmap is already single-channel
    gray_heatmap = heatmap

    # Apply adaptive thresholding to segment the crack
    _, binary_mask = cv2.threshold(gray_heatmap, 100, 255, cv2.THRESH_BINARY)

    # Find contours of the crack
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    segmented_image = img.copy()
    cv2.drawContours(segmented_image, contours, -1, (0, 255, 0), 2)  # Green contours

    return segmented_image


# Function for Gradio prediction
def predict_input_image(image):
    try:
        image = np.array(image)

        # Preprocess image
        image_resized = cv2.resize(image, (224, 224))
        img_array = img_to_array(image_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Get Grad-CAM heatmap
        heatmap, predicted_class = make_gradcam_heatmap(img_array, final_model, last_conv_layer_name)

        # Overlay segmentation
        segmented_image = segment_crack(image_resized, heatmap)

        # Class labels
        CLASS_LABELS = {0: "Surface clean, no cracks found.", 1: "Crack Detected"}
        confidence = final_model.predict(img_array)[0][predicted_class]
        if confidence < 0.2:
            confidence = 1 - confidence
            predicted_class = 1


        return CLASS_LABELS[predicted_class], f"Confidence: {confidence:.2%}", segmented_image

    except Exception as e:
        return "Error", str(e), None

# Setup Gradio Interface
image = gr.Image(type="numpy")
label = gr.Label()
confidence = gr.Textbox()
segmented_output = gr.Image(type="numpy")

inter = gr.Interface(
    fn=predict_input_image,
    inputs=image,
    outputs=[label, confidence, segmented_output],
    title="Crack Detection with Segmentation",
    live=True
)

inter.launch()


